In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Loguru — Logging made easy

https://github.com/Delgan/loguru

In [4]:
from loguru import logger

In [5]:
logger.debug("Hello, cool debugger")

2019-04-23 16:36:06.397 | DEBUG    | __main__:<module>:1 - Hello, cool debugger


# more-itertool

In [4]:
from more_itertools import peekable, chunked

In [3]:
 p = peekable(['a', 'b'])
 p.peek()
 next(p)

'a'

'a'

In [5]:
list(chunked([1, 2, 3, 4, 5, 6], 3))

[[1, 2, 3], [4, 5, 6]]

# MonkeyType
MonkeyType collects runtime types of function arguments and return values, and can automatically generate stub files or even add draft type annotations directly to your Python code based on the types collected at runtime.

# requests-async
Brings support for async/await syntax to Python's fabulous requests library.

# pyright
Static type checker for Python


# black
The uncompromising Python code formatter https://black.readthedocs.io/en/stable/

# Custom Magic CMD and C++ Kernel

In [6]:
from IPython.core.magic import register_cell_magic

In [12]:
@register_cell_magic
def cpp(line, cell):
    """Compile, execute C++ code, and return stdout"""
    # First retrieve the current IPython interpreter instance
    ip = get_ipython()
    
    # Define source and executable filenames
    source_filename = '_temp.cpp'
    program_filename = '_temp'
    
    # write code to C++ file
    with open(source_filename, 'w') as f:
        f.write(cell)
    
    # compile C++ code into an executable
    compile = ip.getoutput('g++ {0:s} -o {1:s}'.format(source_filename, program_filename))
    
    # execute the exectuable and return output
    output = ip.getoutput('./{0:s}'.format(program_filename))
    
    print('\n'.join(output))    

In [13]:
%%cpp
#include <iostream>
int main()
{
    std::cout << "Hello World";
}

Hello World


> Cell magic only exist in runtime
    - pack into extension

```python
# save above cpp() as cpp_ext.py appending below
def load_ipython_extension(ipython):
    ipython.register_magic_function(cpp, 'cell')
```
    - then load with `%load_ext cpp_ext`

### C++ Kernel

In [27]:
%mkdir cpp

In [31]:
%%writefile cpp/cpp_kernel.py

import os
import os.path as op
import tempfile

# import `getoutput()` func by IPython
# allowing sys call from Python
from IPython.utils.process import getoutput

def exec_cpp(code):
    """Compile-execute C++ code and return stdout"""
    # create a temp dir to be deleted end of 'with' context
    with tempfile.TemporaryDirectory() as tmpdir:
        # define source and exec filenames
        source_path = op.join(tmpdir, 'temp.cpp')
        program_path = op.join(tmpdir, 'temp')
        # write code to C++ file
        with open(source_path, 'w') as f:
            f.write(code)
        # compile into exec
        os.system("g++ {0:s} -o {1:s}".format(
            source_path, program_path))
        # execute program and return output
        return getoutput(program_path)

Writing cpp/cpp_kernel.py


In [32]:
%%writefile -a cpp/cpp_kernel.py

"""C++ wrapper kernel."""
from ipykernel.kernelbase import Kernel

class CppKernel(Kernel):
    "Kernel Info"
    implementation = 'C++'
    implementation_version = '1.0'
    language = 'C++'
    language_version = '1.0'
    language_info = {'name': 'c++',
                     'mimetype': 'text/plain'}
    banner = "C++ kernel"
    
    def do_execute(self, code, silent,
                  store_history=True,
                  user_expressions=None,
                  allow_stdin=False):
        """This func called as code cell executed"""
        if not silent:
            # run C++ code and get output
            output = exec_cpp(code)
            # send back result to frontned
            stream_content = {'name': 'stdout',
                             'text': output}
            self.send_response(self.iopub_socket,
                              'stream', stream_content)
            return {'status': 'ok',
                   # base class increments the execution count
                   'execution_count': self.execution_count,
                   'payload': [],
                   'user_expressions': {},
                   }

if __name__ == '__main__':
    from ipykernel.kernelapp import IPKernelApp
    IPKernelApp.launch_instance(kernel_class=CppKernel)

Appending to cpp/cpp_kernel.py


In [33]:
%%writefile cpp/kernel.json

{
    "argv": ["python",
            "cpp/cpp_kernel.py",
            "-f",
            "{connection_file}"
            ],
    "display_name": "C++"
}

Writing cpp/kernel.json


In [34]:
!jupyter kernelspec install --replace --user cpp

[InstallKernelSpec] Installed kernelspec cpp in /Users/Ocean/Library/Jupyter/kernels/cpp


In [35]:
!jupyter kernelspec list

Available kernels:
  cpp          /Users/Ocean/Library/Jupyter/kernels/cpp
  ir           /Users/Ocean/Library/Jupyter/kernels/ir
  julia-0.6    /Users/Ocean/Library/Jupyter/kernels/julia-0.6
  python3      /Users/Ocean/.virtualenvs/general/bin/../share/jupyter/kernels/python3


# Dan Bader Python Tricks INTRO

## PATTERNS FOR CLEANER PYTHON

**ASSERT**
- `assert` best for self-impossibility check
- NOT for **Data Validation** due to security
    - use `if not` and `raise Error('this is erroneous')`

**Complacent Comma Placement**
- _always add comma after elements_

**Context Manager**
- replacing `try...finally...` and more guaranteeing no leaking resources
- setup protocol
    ```python
    # add __enter__ and __exit__ methods
    class ManagedFile:
        def __init__(self, name):
            self.name = name
        def __enter__(self):
            self.file = open(self.name, 'w')
            return self.file
        def __exit__(self, exc_type, exc_val, exc_tb):
            if self.file:
                self.file.close()
    with ManagedFile('hello.txt') as f:
        f.write('hello world')
        f.write('bye now')
    ```
- Another way: using `contextlib.contextmanager` decorator
    ```python
    from contextlib import contextmanager
    @contextmanager
    def managed_file(name):
        try:
            f = open(name, 'w')
            yield f
        finally:
            f.close()
    ```
    - KEY: `managed_file()` is a **GENERATOR** acquires resource firt, then temp-suspends own exec and yields resource so it can be used by caller; when caller leaves `with` context, generator exec so as to release any clean-up

- Pretty API with Context Manager
    - e.g. if "resource" wanted to manage was text indentation levels in some kind of report generator program
    ```python
    # as method
    with Indenter() as indent:
        indent.print('hi!')
        with indent:
            indent.print('hello')
            with indent:
                indent.print('bonjour')
        indent.print('hey')
    # as class
    class Indenter:
        def __init__(self):
            self.level = 0
        def __enter__(self):
            self.level += 1
            return self
        def __exit__(self, exc_type, exc_val, exc_tb):
            self.level -= 1
        def print(self, text):
            print('   ' * self.level + text)
    ```

**Underscores, Dunders, etc**
- 

## Effective Functions

- functions are first-class citizen
- all is objects in python
- functions can be stored as data, passed as data, etc

In [1]:
# Functioanl Programming
# Carrying state of higher-order functions
# LEXICAL CLOSURE

def get_speak_func(text, volume):
    def whisper():
        return text.lower() + '...'
    def yell():
        return text.upper() + '!' 
    if volume > 0.5:
        return yell 
    else:
        return whisper

get_speak_func('Hello, World', 0.7)()

'HELLO, WORLD!'

> A CLOSURE remembers values from its enclosing lexical scope even when program flow is no longer in the scope!!!
    - Practically, meaning not only can func RETURN BEHAVIOURS but also PRE-CONFIGURE those BEHAVIOURS

In [2]:
def make_adder(n):
    def add(x):
        return x + n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)
plus_3(4)
plus_5(4)

9

> `make_adder` serves as FACTORY to create and configure 'adder' func. Note 'adder' func can still access the `n` of `make_adder` (enclosing scope)

**Make Object Callable**
- `__call__` method

In [3]:
class Adder:
    def __init__(self, n):
        self.n = n
    def __call__(self, x):
        return self.n + x
plus_3 = Adder(3)
plus_3(4)

7

**Lambdas Are Single_Expression Functions**
- "expression" is king

In [6]:
(lambda x, y: x + y)(5, 3)

8

> Conceptually, lambda just **inline** of **def**, BUT no need to **BIND** func object to a name before using, simply stated expression then at once eval by calling `()`

In [8]:
# lambda in lexical closure
def make_adder(n):
    return lambda x: x + n

plus_3 = make_adder(3)
plus_3(5)

8

In [10]:
# Harmful
list(filter(lambda x: x % 2 == 0, range(16)))

[0, 2, 4, 6, 8, 10, 12, 14]

In [11]:
# Better
[x for x in range(16) if x % 2 == 0]

[0, 2, 4, 6, 8, 10, 12, 14]

**Power of Decorators**
> **extend and mod the behaviour of callable without permanently mod themselves**
- logging / encorcing access control and authen / instrumentation and timing fucntions / rate-limiting / caching, etc

**how? by wrapping it with closure**

In [12]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() ' 
              f'with {args}, {kwargs}')
        original_result = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}() '
              f'returned {original_result!r}')
        return original_result 
    return wrapper

In [13]:
@trace
def say(name, line):
    return f'{name}: {line}'

In [14]:
say('Jane', 'Hellow, World')

TRACE: calling say() with ('Jane', 'Hellow, World'), {}
TRACE: say() returned 'Jane: Hellow, World'


'Jane: Hellow, World'

**Debugging Decorators**
- Essentially replacing one callable with another
- Pitfall: hiding metadata attached to original callable
- fix: `functools.wraps` used in own decorators to copy over the lost metadata 

> Best practice, wrap all decorators!

In [15]:
import functools

def uppercase(func):
    @functools.wraps(func)
    def wrapper():
        return func().upper()
    return wrapper

In [17]:
@uppercase
def greet():
    """Return a friendly greeting."""
    return 'Hello!'

greet.__name__
greet.__doc__

'Return a friendly greeting.'

## Classes and OOP

In [18]:
class Car:
    def __init__(self, color, mileage):
        self.color = color 
        self.mileage = mileage
    def __repr__(self):
        return '__repr__ for Car'
    def __str__(self):
        return '__str__ for Car'

In [21]:
my_car = Car('red', 37281)
print(my_car)
'{}'.format(my_car)
my_car

__str__ for Car


__repr__ for Car

**Custom Exception**

In [22]:
class NameTooShortError(ValueError): 
    pass
def validate(name):
    if len(name) < 10:
        raise NameTooShortError(name)

In [23]:
validate('jane')

NameTooShortError: jane

In [26]:
class BaseValidationError(ValueError): 
    pass
class NameTooShortError(BaseValidationError): 
    pass
class NameTooLongError(BaseValidationError): 
    pass
class NameTooCuteError(BaseValidationError): 
    pass

try: 
    validate('Jane')
except BaseValidationError as err: 
    handle_validation_error(err)

NameError: name 'handle_validation_error' is not defined

**Cloning Object**

In [27]:
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]] 
ys = list(xs) # Make a shallow copy

xs.append(['new sublist'])
xs, ys

([[1, 2, 3], [4, 5, 6], [7, 8, 9], ['new sublist']],
 [[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [28]:
xs[1][0] = 'X'
xs, ys

([[1, 2, 3], ['X', 5, 6], [7, 8, 9], ['new sublist']],
 [[1, 2, 3], ['X', 5, 6], [7, 8, 9]])

In [36]:
import copy
xs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
zs = copy.deepcopy(xs)
xs[1][0] = 'X'
xs, zs

([[1, 2, 3], ['X', 5, 6], [7, 8, 9]], [[1, 2, 3], [4, 5, 6], [7, 8, 9]])

In [37]:
class Point:
    def __init__(self, x, y):
        self.x = x 
        self.y = y
    def __repr__(self):
        return f'Point({self.x!r}, {self.y!r})'

In [38]:
a = Point(23, 42)
b = copy.copy(a)
a, b, a is b

(Point(23, 42), Point(23, 42), False)

In [39]:
class Rectangle:
    def __init__(self, topleft, bottomright):
        self.topleft = topleft 
        self.bottomright = bottomright
    def __repr__(self):
        return (f'Rectangle({self.topleft!r}, '
                f'{self.bottomright!r})')

In [40]:
rect = Rectangle(Point(0, 1), Point(5, 6))
srect = copy.copy(rect)
rect, srect, rect is srect

(Rectangle(Point(0, 1), Point(5, 6)),
 Rectangle(Point(0, 1), Point(5, 6)),
 False)

In [41]:
rect.topleft.x = 999
rect, srect

(Rectangle(Point(999, 1), Point(5, 6)), Rectangle(Point(999, 1), Point(5, 6)))

In [43]:
drect = copy.deepcopy(srect)
drect.topleft.x = 222
drect, rect, srect

(Rectangle(Point(222, 1), Point(5, 6)),
 Rectangle(Point(999, 1), Point(5, 6)),
 Rectangle(Point(999, 1), Point(5, 6)))

**Abstract Base Classes Keep Inheritance in Check**
- ABC ensure derived calsses use particular methods from base 
- INIT base is impossible and
- forgetting to use interface methods in one of sub-classes raises error as early as possible
- why use `abs` module to solve? 
    - above design is pretty common 

In [44]:
class Base:
    def foo(self):
        raise NotImplementedError()
    def bar(self):
        raise NotImplementedError()
class Concrete(Base): 
    def foo(self):
        return 'foo() called'
    # Oh no, we forgot to override bar()...
    # def bar(self):
    #     return "bar() called"

In [45]:
b = Base()
b.foo()

NotImplementedError: 

In [48]:
c = Concrete()
c.foo()

'foo() called'

In [49]:
c.bar()

NotImplementedError: 

In [50]:
# Updated using ABC
from abc import ABCMeta, abstractmethod

class Base(metaclass=ABCMeta): 
    @abstractmethod
    def foo(self): 
        pass
    @abstractmethod
    def bar(self): 
        pass

class Concrete(Base): 
    def foo(self):
        pass
    # We forget to declare bar() again...

In [51]:
assert issubclass(Concrete, Base)

In [52]:
# yet another benefit: subclasses of Base raise TypeError
# at INIT time whenever forgetting to use any abstract methods

c = Concrete()

TypeError: Can't instantiate abstract class Concrete with abstract methods bar

> Without `abc`, we'd only get `NotImplementedError` if missing method actually called; INIT time raise is good - harder to write invalid subclasses
    1. ABC ensure that derived classes implement particular methods from base class at INIT time
    2. Using ABC can help avoid bugs and make class hierarchies easier to maintain

**Namedtuple**
- view them as extension of built-it `tuple`
- `tuple` immutable container of arbitrary objects

In [53]:
tup = ('hello', object(), 42)
tup, tup[2]

(('hello', <object at 0x12393e480>, 42), 42)

In [54]:
tup[2] = 23

TypeError: 'tuple' object does not support item assignment

> Pitfall: only able to pull out via accessing integer index

In [56]:
from collections import namedtuple

Car = namedtuple('Car', 'color mileage')

# Or more clearly

Car = namedtuple('Car', [
    'color',
    'mileage',
])

> The class name is used in the docstring and the `__repr__` implementation that namedtuple auto- matically generates for us.

In [57]:
my_car = Car('red', 3812.4)
my_car.color, my_car.mileage

('red', 3812.4)

In [58]:
tuple(my_car)

('red', 3812.4)

In [59]:
print(*my_car)

red 3812.4


In [60]:
my_car # free string repr for free

Car(color='red', mileage=3812.4)

> Subclassing Namedtuples
    - Extend its class like any other class and add methods and attributes

In [61]:
class MyCarWithMethods(Car): 
    def hexcolor(self):
        if self.color == 'red': 
            return '#ff0000'
        else:
            return '#000000'

In [62]:
c = MyCarWithMethods('red', 1234)
c.hexcolor()

'#ff0000'

> HOWEVER: For example, adding a new immutable field is tricky because of how namedtuples are structured internally. The easiest way to create hier- archies of namedtuples is to use the base tuple’s _fields property:

In [63]:
ElectricCar = namedtuple(
    'ElectricCar', Car._fields + ('charge',))
ElectricCar('red', 1234, 45.0)

ElectricCar(color='red', mileage=1234, charge=45.0)

In [64]:
# _asdict() returns contents as dict
my_car._asdict()

OrderedDict([('color', 'red'), ('mileage', 3812.4)])

In [65]:
# great for avoiding typos in field names when generating
# JSON output
import json
json.dumps(my_car._asdict())

'{"color": "red", "mileage": 3812.4}'

In [66]:
# _replace creats shallow copy for seletively
my_car._replace(color='blue')

Car(color='blue', mileage=3812.4)

In [67]:
# _make() classmethod for creating new instances from seq or iter
Car._make(['red', 999])

Car(color='red', mileage=999)

> WHEN to USE?
    1. easy way to clean up code and make it more readable 
    2. e.g. from ad-hoc dict with fixed format to namedtuples helps express intentions more clearly
    3. somewhat "self-documenting" than dict or tuple

**Class vs Instance Variable Pitfalls**
- mod class variables instantly mod all instances inherited class variable
- shadowing class var name via instances is a pitfall of OOP of python

In [68]:
# Example of using class variable in instances
class CountedObject: 
    num_instances = 0
    
    def __init__(self): 
        self.__class__.num_instances += 1

CountedObject.num_instances

0

In [69]:
CountedObject().num_instances

1

In [70]:
CountedObject().num_instances

2

In [71]:
CountedObject().num_instances

3

In [72]:
CountedObject.num_instances

3

In [73]:
# Potential mistake had one written constructors as follows:
# WARNING: This implementation contains a bug 

class BuggyCountedObject:
    num_instances = 0
    
    def __init__(self): 
        self.num_instances += 1 # !!!

BuggyCountedObject.num_instances

0

In [74]:
BuggyCountedObject().num_instances

1

In [75]:
BuggyCountedObject().num_instances

1

In [76]:
BuggyCountedObject().num_instances

1

In [77]:
BuggyCountedObject.num_instances

0

> ERROR: **shadowed** the `num_instance` **class variable** by creating an instance variable of the **sanme name in constructor**
    - storing result in instance varibale - other instances never see updated value
    - double check scoping dealing with **shared state** on a class

**Instance, Class, Static Methods**

In [78]:
class MyClass:
    def method(self):
        return 'instance method called', self
    
    @classmethod
    def classmethod(cls):
        return 'class method called', cls
    
    @staticmethod
    def staticmethod():
        return 'static method called'

1. Instance Methods
    - `method` a regular instance method with `self` pointing to an instance of `MyClass` on called
    - Through `self` parameter instance method can freely access attributes and other methods on the same object - giving lots of power in moding object's state
    - Furthermore, instance methods can access class via `self.__class__` attribute - mod class state !!
2. Class Methods
    - `classmethod` marked with decorator to flag it
    - instead of `self`, it take `cls` parameter pointing to class - NOT the OBJECT INSTANCE - when called
    - CANNOT mod object instance state but still class state applied across all instances of the class
3. Static Methods
    - Neither `self` nor `clf`, though it can be made to accept an arbitrary number of other parameters
    - Cannot mod object state or class state - restricted in what data they can access
    - Primarily a way to **namespace** methods!!

In [79]:
obj = MyClass()
obj.method()

('instance method called', <__main__.MyClass at 0x123d31da0>)

In [80]:
# manual instantiation
MyClass.method(obj)

('instance method called', <__main__.MyClass at 0x123d31da0>)

In [82]:
obj.__class__.classmethod # accessing class state !!!

<bound method MyClass.classmethod of <class '__main__.MyClass'>>

In [83]:
# class method
obj.classmethod()

('class method called', __main__.MyClass)

In [84]:
MyClass.classmethod()

('class method called', __main__.MyClass)

In [85]:
# static method
obj.staticmethod()

'static method called'

In [87]:
# Example
class Pizza:
    def __init__(self, ingredients):
        self.ingredients = ingredients 
    def __repr__(self):
        return f'Pizza({self.ingredients!r})'

In [88]:
Pizza(['cheese', 'tomatoes'])

Pizza(['cheese', 'tomatoes'])

In [89]:
# make factory class
class Pizza:
    def __init__(self, ingredients):
        self.ingredients = ingredients 
    def __repr__(self):
        return f'Pizza({self.ingredients!r})'
    @classmethod
    def margherita(cls):
        return cls(['mozzarella', 'tomatoes'])
    @classmethod
    def prosciutto(cls):
        return cls(['mozzarella', 'tomatoes', 'ham'])

In [90]:
Pizza.margherita()

Pizza(['mozzarella', 'tomatoes'])

In [91]:
Pizza.prosciutto()

Pizza(['mozzarella', 'tomatoes', 'ham'])

> Use factory func to create new Pizza objects configured at will
    - ALL use the same `__init__` internallly and provide shortcut for remmebering all various ingredients
    - View them as allowing to define alternative constructors of classes
    - Using classmethod makes it possible to add as many alternative constructors as necessary - make interface for classes self-documenting and simplify usage

In [93]:
# staticmethod example
import math
class Pizza:
    def __init__(self, radius, ingredients):
        self.radius = radius 
        self.ingredients = ingredients
    def __repr__(self):
        return (f'Pizza({self.radius!r}, '
                f'{self.ingredients!r})') 
    def area(self):
        return self.circle_area(self.radius)
    @staticmethod
    def circle_area(r):
        return r ** 2 * math.pi

> First mod constructor and `__repr__` to accept extra radius arg
    - `area()` instance method - good candidate for `@property`
    - instead of calcu area directly within `area()`, by using circle area formula, factored out to a separate staticmethod

In [94]:
p = Pizza(4, ['mozzarella', 'tomatoes'])
p

Pizza(4, ['mozzarella', 'tomatoes'])

In [112]:
p.area()

50.26548245743669

In [97]:
Pizza.circle_area(4)

50.26548245743669

> Benefits of staticmethod
    1. limitation is great signal to show particular method is independent from all else - `circle_area()` cannot mod class/instance
    2. test code dev

## Common Data Structures

In [4]:
# Dictionaries
import collections

# prefer explicit orderDict if enforcing
d1 = collections.OrderedDict(one=1, two=2, three=3)
d1
# default creation
d2 = collections.defaultdict(list)
d2['dogs'].append('Rufus')
d2['dogs'].append('Kathrin')
d2
# ChainDict
d3 = collections.ChainMap({'one': 1,
                          'two': 2},
                         {'three': 3,
                         'four': 4})
d3['one']

OrderedDict([('one', 1), ('two', 2), ('three', 3)])

defaultdict(list, {'dogs': ['Rufus', 'Kathrin']})

1

In [5]:
# types.MappingProxyType - A Wrapper for Making 
# Read-Only Dict
# used to create immutable proxy versions of dict
# useful if returning dict carrying internal state
# from a class or module, while discouraging write
# access to this object

from types import MappingProxyType

writable = {'one': 1, 'two': 2}
read_only = MappingProxyType(writable)

read_only['one']

read_only['one'] = 23

1

TypeError: 'mappingproxy' object does not support item assignment

**ARRAY**
- fixed sized data records allowing each elem effi loc based on its index
- adjoining blocks of MEM - _contiguous_ data structures - as opposed to _linked_ like list

**List - mutable dynamic array**
- nice `__repr__`, holds arbitrary data types

**Tuple - immutable containers**
- defined at creation
- arbitrary types
- also less tightly packed than **typed array**

**`array.array` - basic typed array**
- C-style data types like `bytes, int32, float...`

In [7]:
import array

arr = array.array('f', (1.0, 1.5, 2.0, 2.5))
arr
del arr[1]
arr
arr.append(42.0)
arr
arr[1] = 'hello'

array('f', [1.0, 1.5, 2.0, 2.5])

array('f', [1.0, 2.0, 2.5])

array('f', [1.0, 2.0, 2.5, 42.0])

TypeError: must be real number, not str

**str - immutable arrays of Unicode Char**

In [11]:
arr = 'abcd'
arr[1]
arr
# arr[1] = 'e'
    # str object immutable
# del arr[1] 
    # TypeError: str object cannot del

# strings can be unpacked into a list 
# to get a mutable repr
list(arr)
''.join(list('abcd'))

# strings recursive data struct
type('abc')
type('abc'[0])


'b'

'abcd'

['a', 'b', 'c', 'd']

'abcd'

str

str

**bytes - Immutable Arrays of Single Bytes**

- or integers in range of [0, 255]
- conceptually similar to str
- as if immutable arrays of bytes
- unlike str, dedicated bytearray into which be unpacked

In [13]:
arr = bytes((0, 1, 2, 3))
arr[1]
arr

arr = bytearray((0, 1, 2, 3))
# mutable
arr[1]
arr
arr[1] = 23
arr

1

b'\x00\x01\x02\x03'

1

bytearray(b'\x00\x01\x02\x03')

bytearray(b'\x00\x17\x02\x03')

**KEY**

> **Storing arbitray -> `list, tuple`**

> **Numeric performant -> `array.array`**

> **Textual Unicdoe -> `str` or `list` of char**

> **Contiguous bytes -> `bytes, bytearrays`**

**RECORDS, STRUCTS, DATA TRANSFER OBJECTS**

In [14]:
# dict skipped

# tuple - slightly lighter and also faster than list
import dis

dis.dis(compile("(23, 'a', 'b', 'c')", '', 'eval'))

  1           0 LOAD_CONST               0 ((23, 'a', 'b', 'c'))
              2 RETURN_VALUE


In [15]:
dis.dis(compile("[23, 'a', 'b', 'c']", '', 'eval'))

  1           0 LOAD_CONST               0 (23)
              2 LOAD_CONST               1 ('a')
              4 LOAD_CONST               2 ('b')
              6 LOAD_CONST               3 ('c')
              8 BUILD_LIST               4
             10 RETURN_VALUE


In [16]:
# best: namedtuple
import collections
from sys import getsizeof

tup1 = collections.namedtuple('Point', 'x y z')(1, 2, 3)
tup2 = (1, 2, 3)

getsizeof(tup1)
getsizeof(tup2)

72

72

**`typing.NamedTuple` - Improved Namedtuples**
- updated syntax for defining new record types and type hints
- enforced by `mypy`

In [18]:
from typing import NamedTuple

class Car(NamedTuple): 
    color: str
    mileage: float
    automatic: bool

car1 = Car('red', 3812.4, True)
# Instances have a nice repr:
car1

# Accessing fields:
car1.mileage
# car1.mileage = 12
    # AttributeError: can't set

# Type annotations not enforced without
# separate chceking tool
Car('red', 'NOT_A_FLOAT', 99)

Car(color='red', mileage=3812.4, automatic=True)

3812.4

Car(color='red', mileage='NOT_A_FLOAT', automatic=99)

**`struct.Struct` - Serialised C Structs**
- converts between Python val and C-structs serialised into Python `bytes` objects
- to handle binary data stored in files or coming in from network
- C types `char, int, long, unsigned`
- intended primarily as data exchange format rather than container
- most cases packing primitive data into sructs may use less MEM but would be quite an advanced opt

In [19]:
from struct import Struct

MyStruct = Struct('i?f')
data = MyStruct.pack(23, False, 42.0)
data

MyStruct.unpack(data)

b'\x17\x00\x00\x00\x00\x00\x00\x00\x00\x00(B'

(23, False, 42.0)

**`types.SimpleNamespace` - Fancy Attribute Access**
- "esoteric" choice for attribute access to its namespace
- expose all keys as class attributes
- `obj.key` 'dotted' attribute access instead of `obj['key']` indexing
- `__repr__` by default
- simple, basically glorified dict 

In [20]:
from types import SimpleNamespace

car1 = SimpleNamespace(color='red',
                       mileage=3812.4,
                       automatic=True)
car1
car1.mileage = 12
car1.windshield = 'broken'
del car1.automatic
car1

namespace(automatic=True, color='red', mileage=3812.4)

namespace(color='red', mileage=12, windshield='broken')

**IN SUM**

> **Only 2-3 fields => plain tuple**

> **Immutable fields => plain tuple, `collections.namedtuple` and `typing.NamedTuple`**

> **Lock down field names to avoid typos => above latter two**

> **Keep simple => plain dict due to JSON-like syntax**

> **Full control over data structure => custom class with `@property` setters and getters**

> **Add behaviour/methods => custom class from zero or extending two namedtuples**

> **Pack data tightly to serilise to disk or networking => `struct.Struct`**

> **safe default => `typing.NamedTuple` for plain record, struct or data object

**SETS AND MULTISETS**
- syntactic suger

In [22]:
vowels = {'a', 'e', 'i', 'o', 'u'}
vowels
squares = {x * x for x in range(10)}
squares

# BEWARE empty set
set()

{'a', 'e', 'i', 'o', 'u'}

{0, 1, 4, 9, 16, 25, 36, 49, 64, 81}

set()

In [24]:
vowels = {'a', 'e', 'i', 'o', 'u'}
'e' in vowels
letters = set('alice')
letters.intersection(vowels)
vowels.add('x')
vowels
len(vowels)

True

{'a', 'e', 'i'}

{'a', 'e', 'i', 'o', 'u', 'x'}

6

**`frozenset` - Immutable Set**

- static and only allow query ops 
- NO inserts or deletions
- static, hashable to use as keys

In [25]:
vowels = frozenset({'a', 'e', 'i', 'o', 'u'})
# vowels.add('p')
    # AttributeError:

# Frozensets are hashable and can
# be used as dictionary keys:
d = { frozenset({1, 2, 3}): 'hello' }
d[frozenset({1, 2, 3})]

'hello'

**`collections.Counter` - Multisets**
- or bag type for multi-occurrence

In [26]:
from collections import Counter
inventory = Counter()
loot = {'sword': 1, 'bread': 3}
inventory.update(loot)
inventory

more_loot = {'sword': 1, 'apple': 1}
inventory.update(more_loot)
inventory

Counter({'sword': 1, 'bread': 3})

Counter({'sword': 2, 'bread': 3, 'apple': 1})

In [27]:
# BEWARE of counting
len(inventory)

# Total No. of elements
sum(inventory.values()) 

3

6

**STACK (LIFEO)**
- no random access
- `push` and `pop`
- similar to queues, linear colletions differing in order accessed (queue = FIFO)
- Runtime MEM "stack" use short-beautiful algo => **depth-first search** (DFS) on a tree or grpah data struct

**`list` simple, built-in stack**
- O(1) time push and pop
- downside: less consistent than O(1) by linked-list like `collections.deque`) but fast random access 
- **to get amortised O(1) insert/delete, new items must be added to end with `append()` and remove `pop()`**

**`collections.deque` - Fast & Robust Stacks**
- double-ended queue adding/removing in O(1) either end
- serving as queue and stack
- douhbly-linked lists great insistent

In [28]:
from collections import deque
s = deque()
s.append('eat')
s.append('sleep')

s
s.pop()
s.pop()

deque(['eat', 'sleep'])

'sleep'

'eat'

**`queue.LifoQueue` - Locking Semantics for Parallel Compu**
- locking semantics for multiple concurrent producers and consumers

> **for non-parallel:**
    1. `list` backed by dynamic array great and fast random access but may need resizing when added or removed; over-allocates backing storage so as not to push or pop resizing but beware `append` and `pop`
    2. `collections.deque` backed by douhbly-linked list front and end O(1) but slower in middle; easier to use as no "wrong end"; best for LIFO queue

**QUEUE (FIFO)**
- enqueue and dequeue
- NO random access like pipe
- great for BFS algo 
- great for priority queuing 
- **list `pop(0)` is SLOW**

In [30]:
from collections import deque

q = deque()
q.append('eat')
q.append('sleep')
q.append('code')

q
q.popleft()
q.pop()

deque(['eat', 'sleep', 'code'])

'eat'

'code'

**`queue.Queue` - Locking Semantics for Parallel Compu**
- synchronised locking for concurrent maker and user
- might be helpful or just overhead

In [32]:
from queue import Queue

q = Queue()
q.put('eat')
q.put('sleep')
q.put('code')
q
q.get()
q.get()
q.get_nowait()

'eat'

'sleep'

'code'

**`multiprocessing.Queue` - Shared Job Queues**
- allow queued items processed in parallel 
- sharing data across procs, this makes it easy 
- can store and transfer any pickle-able across procs 

In [33]:
from multiprocessing import Queue

q = Queue()
q.put('eat')
q

**PRIORITY QUEUE**
- totally-ordered like weight value
- retrieval by order/rank
- used for scheduling / urgency
- `list` manually sorted queue - sorted list BUT insertion = O(n) while O(log n) using `bisect.insort` always slow
- only suitable as priority queue when FEW INSERTIONS

In [34]:
q = []

q.append((2, 'code'))
q.append((1, 'eat'))
q.append((3, 'sleep'))

# NOTE beware to re-sort every time insertion
q.sort(reverse=True)

while q:
    next_item = q.pop()
    print(next_item)

(1, 'eat')
(2, 'code')
(3, 'sleep')


**`heapq` LIST-based Binary heaps**
- O(log n) insertion/extraction of smallest element
- good for priority queues in Python as only min-heap
- extra steps need to ensure sort stability and other pracctical features

In [35]:
import heapq

q = []

heapq.heappush(q, (2, 'code'))
heapq.heappush(q, (1, 'eat'))
heapq.heappush(q, (3, 'sleep'))
while q:
    next_item = heapq.heappop(q)
    print(next_item)


(1, 'eat')
(2, 'code')
(3, 'sleep')


**`queue.PriorityQueue` Beautiful PQ**
- use `heapq` internally and shares the same time and space comlexities
- BUT synchronised and locking semantics for concurrent maker and user

In [38]:
from queue import PriorityQueue 

q = PriorityQueue()
q.put((2, 'code'))
q.put((1, 'eat'))
q.put((3, 'sleep'))

while not q.empty(): 
    next_item = q.get()
    print(next_item)


(1, 'eat')
(2, 'code')
(3, 'sleep')


## Loop and Iteration

In [40]:
# C style
my_items = ['a', 'b', 'c']
i=0
while i < len(my_items):
    print(my_items[i])
    i += 1

a
b
c


In [41]:
for i in range(len(my_items)): 
    print(my_items[i])

a
b
c


In [42]:
# pythonic
for item in my_items: 
    print(item)

a
b
c


In [43]:
for i, item in enumerate(my_items):
    print(f'{i}: {item}')

0: a
1: b
2: c


In [44]:
emails = {
    'Bob': 'bob@example.com',
    'Alice': 'alice@example.com',
}

for name, email in emails.items():
    print(f'{name} -> {email}')

Bob -> bob@example.com
Alice -> alice@example.com


> C-style loop

```python
for (int i = a; i < n; i += s) { 
    // ...
}

# pythonic
 for i in range(a, n, s):
        # ...

```

**Comprehensions**
- list
- set
- dictionary

In [45]:
{ x: x * x for x in range(5) }

{0: 0, 1: 1, 2: 4, 3: 9, 4: 16}

**Iterators**
- backed by `__iter__` and `__next__` dunder

In [55]:
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return RepeaterIterator(self)

class RepeaterIterator:
    # LINK to Repeater
    def __init__(self, source):
        self.source = source
    
    # REACH back into "source" Repeater
    def __next__(self):
        return self.source.value

> What going on `for-loop`

```python
# INF loop
repeater = Repeater('Hello')
iterator = repeater.__iter__()
while True:
    item = iterator.__next__()
    print(item)
```

1. first prepared repeater for iteration calling its `__iter__` method returning actual `iterator` object
2. then loop repeatedly called iterator objects's `__next__` method to retrieve values from it

> **Similar to database cursors, init cursor and prepare for reading, then fetch data from it into local variables as needed one-by-one**

> **NEVER MORE THAN ONE IN FLIGHT, hence MEM-efficient and Repeater is INFINITE sequence of elements able to iterate BUT LIST would be doomed**

> Manual emulation

In [60]:
iterator = iter(repeater)
next(iterator)
next(iterator)
next(iterator)

'Hello'

'Hello'

'Hello'

> **syntactic suger `iter()` for `obj.__iter__()` similar to `len(x)` == `x__len__`**

> **WHY NEED `RepeaterIterator` class? Need it to HOST `__next__` for fetching new values from iterator - no matter WHERE it defined BUT returns ANY object**

> **IDEA: `RepeaterIterator` returns same value over no keeping track of any internal state, what if adding `__next__` directly to `Repeater`?**

In [61]:
class Repeater:
    def __init__(self, value):
        self.value = value 
    def __iter__(self):
        return self
    def __next__(self): 
        return self.value

> `StopIteration` error raised at exhausion - normally cannot be RESET - iterate anew need to request fresh iterator obj with `iter()` func

In [63]:
# stops iter after set num of repeat
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value

In [64]:
repeater = BoundedRepeater("Hello", 3)
for item in repeater:
    print(item)

Hello
Hello
Hello


In [65]:
# strip syntactic suger
repeater = BoundedRepeater('Hello', 3)
iterator = iter(repeater)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        break
    print(item)

Hello
Hello
Hello


> Each time `next()` called, check for exception and break while loop if needed; being able to write 3-line for-in loop instead of 8-line while loop is quite nice easier and why iterator in python powerful

1. Iterators provide seq interface MEM-efficient and pythonic
2. Support iteration obj needs to implement `iterator protocol` by providing `__iter__` and `__next__`
3. Class-based iterators only one way to write iterable obj; also consider generators and generator exp

**GENERATOR - simplified iterator**
- above lots of boilerplate
- Inifte generators

In [66]:
def repeater(value):
    while True:
        yield value

```python
for x in repeater('Hi'):
    print(x)
```

> Still inf loop but much shorter

1. Generator not really func, merely create-return generator object!

In [68]:
repeater('Hey')

<generator object repeater at 0x12ec4a4f8>

> Code in generator func only exec when `next()` is called on generator object:

In [69]:
generator_obj = repeater('Hey')
next(generator_obj)

'Hey'

> **stopper yield, `return` invoked inside func, permanently passing control back to caller whereas `yield` passing control back TEMPORARILY**

> **`return` disposes func's local state, `yield` suspends func and retains its local state!!! Meaning local variables and exec state of generator func are ONLY STASHED AWAY TEMPORARILY and NOT DISCAEDED FULLY - resumed at any time by calling `next()`**

> Most types of iterators, writing a generator func easier and more legible than defining long-winded class-based iterator

**Generators Stop Generating**
- Base calss iterator stoped with `StopIteration` exception
- Generator's fully compatible implement this underneath
- Nice interface meaning no longer have to worry about raising error at all as gen stops as soon as control flow returns from gen func by any means other than a `yield` statement

In [70]:
# Example
def repeat_three_times(value):
    yield value
    yield value
    yield value

In [71]:
# see how it ends
for x in repeat_three_times('Hey there'):
    print(x)

Hey there
Hey there
Hey there


> Gen stoped making after 3 yields, assuming it did so by raising exception when exec reaching end

In [72]:
# Let's confirm
iterator = repeat_three_times('Hey there')
next(iterator)
next(iterator)
next(iterator)
next(iterator)

'Hey there'

'Hey there'

'Hey there'

StopIteration: 

> This iterator behaved just like expected by raising exception

> Revisit iterator previously defined

In [80]:
# mod it as generator
def bounded_repeater(value, max_repeats):
    count = 0
    while True:
        if count >= max_repeats:
            return
        count += 1
        yield value

> Intentionally made `while` loop little unwieldy showing how invoking `return` from generator causing iteration to stop with exception

In [81]:
for x in bounded_repeater('Hi', 4):
    print(x)

Hi
Hi
Hi
Hi


> Now a gen stops after configurable rep using `yield` to pass back values till hitting `return` 

In [82]:
# Further simplify taking IMPLICIT 
# return NONE end of all func
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

> 12-line class-based iterator to 3-line generator having the same functionality - 75% reduction 

> generator abstract away most of boilerplate in class-based iterators 

**GENERATOR EXPRESSION**
- "syntactic suger" recurring theme
- class-based iterators and generator func are twoexpressions of the same underlying design pattern
- Gen a shortcut for supporting iterator protocol avoiding verbosity in class-based 
- gen-expr another sugar more effective shortcut for writing iterators 

In [83]:
iterator = ('Hello' for i in range(3))

In [84]:
# 1-line == 3-line
for x in iterator:
    print(x)

Hello
Hello
Hello


> One small caveat: once gen expr consumed, it cannot be restarted or reused - beware

**In-Line Gen-Epxr**

In [85]:
for x in ('Bom dia' for i in range(3)):
    print(x)

Bom dia
Bom dia
Bom dia


In [86]:
# Drop () if used as arg to func
sum(x * 2 for x in range(10))

90

**ITERATOR CHAINS**

> make series of int from 1 to 8 keeping running counter yielding new value on next()

In [89]:
def integers():
    for i in range(1, 9):
        yield i

In [90]:
chain = integers()
list(chain)

[1, 2, 3, 4, 5, 6, 7, 8]

> Generators can be CONNECTED to each other to build efficient DATA PROCESSING algo pipeline!!

In [91]:
def squared(seq):
    for i in seq:
        yield i * i

In [92]:
chain = squared(integers())
list(chain)

[1, 4, 9, 16, 25, 36, 49, 64]

In [93]:
def negated(seq):
    for i in seq:
        yield -i

In [94]:
chain = negated(squared(integers()))
list(chain)

[-1, -4, -9, -16, -25, -36, -49, -64]

In [95]:
# Shrink down pipeline even more
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)
negated
list(negated)

<generator object <genexpr> at 0x12ec4a840>

[0, -1, -4, -9, -16, -25, -36, -49]

## Dictionary Trick

In [96]:
# query with default error
name_for_userid = {
    382: 'Alice',
    950: 'Bob',
    590: 'Dilbert',
}

def greeting(userid):
    return 'Hi %s!' % name_for_userid.get(userid, 'there')

greeting(950)
greeting(33333)

'Hi Bob!'

'Hi there!'

**Sorting Dict for Fun**

In [97]:
xs = {'a': 4, 'c': 2, 'b': 3, 'd': 1}

sorted(xs.items())

[('a', 4), ('b', 3), ('c', 2), ('d', 1)]

In [98]:
sorted(xs.items(), key=lambda x: x[1])

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

> Replacing lambda-based index lookup with `operator.itemgetter`

In [102]:
import operator

sorted(xs.items(), key=operator.itemgetter(1))

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

In [103]:
# Lambda finer control
sorted(xs.items(), key=lambda x: abs(x[1]))

[('d', 1), ('c', 2), ('b', 3), ('a', 4)]

**Emulating Switch/Case with Dict**

In [104]:
def myfunc(a, b):
    return a + b

funcs = [myfunc]
funcs[0]

<function __main__.myfunc(a, b)>

In [105]:
funcs[0](2, 3)

5

**define a dict mapping lookup keys for input conditions to func carrying out intended ops**

```python
func_dict = {
    'cond_a': handle_a,
    'cond_b': handle_b
}

cond = 'cond_a'
func_dict[cond]()

# Better way
func_dict.get(cond, handle_default)()
```

**Another example**

In [107]:
def dispatch_if(operator, x, y):
    if operator == 'add': 
        return x + y
    elif operator == 'sub': 
        return x - y
    elif operator == 'mul': 
        return x * y
    elif operator == 'div': 
        return x / y

dispatch_if('mul', 2, 8)

# Transform into new func using dictionary map opcodes 
def dispatch_dict(operator, x, y):
    return {
        'add': lambda: x + y, 
        'sub': lambda: x - y, 
        'mul': lambda: x * y, 
        'div': lambda: x / y,
    }.get(operator, lambda: None)()

dispatch_dict('mul', 2, 8)

16

16

**Craziest Dict Expr in the West**

In [108]:
{True: 'yes', 1: 'no', 1.0: 'maybe'}

{True: 'maybe'}

In [109]:
# Because...
xs = dict()
xs[True] = 'yes'
xs[1] = 'no'
xs[1.0] = 'maybe'

In [110]:
True == 1 == 1.0

True

In [111]:
bool(1), bool(0)

(True, False)

**Investigator non-updating keys - hash collision?**

In [113]:
class AlwaysEquals:
    def __eq__(self, other):
        return True
    def __hash__(self): 
        return id(self)

AlwaysEquals() == 'waaat?'

True

In [114]:
# Return unique hash value generated by 
# built-in id() function
objects = [AlwaysEquals(),
          AlwaysEquals(),
          AlwaysEquals(),]
[hash(obj) for obj in objects]

[5091557272, 5091556768, 5091554976]

**CPython, `id()` returns address of the object in MEM, guaranteed unique**

In [115]:
# Keys NOT overriwtten even though
# always equal
{AlwaysEquals(): 'yes', AlwaysEquals(): 'no'}

{<__main__.AlwaysEquals at 0x12f7afb00>: 'yes',
 <__main__.AlwaysEquals at 0x10c961080>: 'no'}

In [116]:
# see if returning same hash value 
# is enough to cause keys to be overwritten
class SameHash:
    def __hash__(self):
        return 1
    
a = SameHash()
b = SameHash()
a == b

hash(a), hash(b)

False

(1, 1)

In [117]:
# See how dict react when use SameHash as dict key
{a: 'a', b: 'b'}

{<__main__.SameHash at 0x12f7dfcc0>: 'a',
 <__main__.SameHash at 0x12f7dfb00>: 'b'}

> SO Dict check for equality and compare hash value to determine if 2 keys the same

> **The crazy dict eval coz keys True, 1 and 1.0 all equal AND they all have the same hash value:**

In [118]:
(hash(True), hash(1), hash(1.0))

(1, 1, 1)

> Dict treat keys as identical if `__eq__` comparison True AND `hash()` values the same

**So Many Ways to Merge Dict**
- Need a way to merge 2 or more dict into one so as to have combo of keys and values of source dicts

In [119]:
xs = {'a': 1, 'b': 2}
ys = {'b': 3, 'c': 4}

In [120]:
# Similar to chain update()
zs = {**xs, **ys}
zs

{'a': 1, 'b': 3, 'c': 4}

**Pretty Print Dict**

In [121]:
mapping = {'a': 23, 'b': 42, 'c': 0xc0ffee}
str(mapping)

"{'a': 23, 'b': 42, 'c': 12648430}"

In [122]:
import json
json.dumps(mapping, indent=4, sort_keys=True)

'{\n    "a": 23,\n    "b": 42,\n    "c": 12648430\n}'

In [124]:
import pprint
pprint.pprint(mapping)

{'a': 23, 'b': 42, 'c': 12648430}


## Pythonic Productive

**Exploring Modules and Objects**

In [125]:
import datetime
dir(datetime)

['MAXYEAR',
 'MINYEAR',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'date',
 'datetime',
 'datetime_CAPI',
 'sys',
 'time',
 'timedelta',
 'timezone',
 'tzinfo']

In [126]:
dir(datetime.date)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'ctime',
 'day',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'min',
 'month',
 'replace',
 'resolution',
 'strftime',
 'timetuple',
 'today',
 'toordinal',
 'weekday',
 'year']

In [127]:
[_ for _ in dir(datetime) if 'date' in _.lower()]

['date', 'datetime', 'datetime_CAPI']

**Bytecode**

In [129]:
def greet(name):
    return 'Hello, ' + name + '!'

In [131]:
greet.__code__.co_code

b'd\x01|\x00\x17\x00d\x02\x17\x00S\x00'

In [132]:
greet.__code__.co_consts

(None, 'Hello, ', '!')

In [133]:
greet.__code__.co_varnames

('name',)

In [134]:
import dis
dis.dis(greet)

  2           0 LOAD_CONST               1 ('Hello, ')
              2 LOAD_FAST                0 (name)
              4 BINARY_ADD
              6 LOAD_CONST               2 ('!')
              8 BINARY_ADD
             10 RETURN_VALUE
